In [4]:
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import Adam
from keras.datasets import mnist
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report

In [5]:
LEARNING_RATE = 0.001
EPOCHS = 10
BATCH_SIZE = 256

In [6]:
def create_model(width, height, depth, classes):
    model = Sequential()
    inputShape = (height, width, depth)

    model.add(Conv2D(64, kernel_size=(5, 5), activation='relu', padding="same", input_shape=inputShape))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding="same"))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding="same"))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(256))
    model.add(Activation("relu"))
    model.add(Dropout(0.3))

    model.add(Dense(128))
    model.add(Activation("relu"))
    model.add(Dropout(0.2))

    model.add(Dense(classes))
    model.add(Activation("softmax"))

    adam = Adam(learning_rate=LEARNING_RATE)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

In [7]:
model = create_model(width=28, height=28, depth=1, classes=10)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 64)        1664      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 7, 7, 128)         73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 3, 3, 128)        0

In [8]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1))

X_train = X_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0

lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_test = lb.transform(y_test)

In [9]:
model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(X_test, y_test), verbose=1)

Epoch 1/10
235/235 [==============================] - 45s 188ms/step - loss: 0.3220 - accuracy: 0.8968 - val_loss: 0.0602 - val_accuracy: 0.9806
Epoch 2/10
235/235 [==============================] - 42s 179ms/step - loss: 0.0765 - accuracy: 0.9769 - val_loss: 0.0323 - val_accuracy: 0.9888
Epoch 3/10
235/235 [==============================] - 42s 181ms/step - loss: 0.0514 - accuracy: 0.9847 - val_loss: 0.0303 - val_accuracy: 0.9895
Epoch 4/10
235/235 [==============================] - 45s 192ms/step - loss: 0.0414 - accuracy: 0.9880 - val_loss: 0.0250 - val_accuracy: 0.9913
Epoch 5/10
235/235 [==============================] - 42s 178ms/step - loss: 0.0324 - accuracy: 0.9899 - val_loss: 0.0263 - val_accuracy: 0.9910
Epoch 6/10
235/235 [==============================] - 42s 179ms/step - loss: 0.0273 - accuracy: 0.9919 - val_loss: 0.0268 - val_accuracy: 0.9908
Epoch 7/10
235/235 [==============================] - 44s 188ms/step - loss: 0.0238 - accuracy: 0.9926 - val_loss: 0.0250 - val_ac

In [12]:
from keras.models import save_model

save_model(model, 'digit_classifier.h5')

In [ ]:
from keras.models import load_model

model = load_model('digit_classifier.h5')

In [14]:
predictions = model.predict(X_train)
print(classification_report(y_train.argmax(axis=1), predictions.argmax(axis=1), target_names=[str(x) for x in lb.classes_]))

1875/1875 [==============================] - 22s 12ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5923
           1       1.00      1.00      1.00      6742
           2       1.00      1.00      1.00      5958
           3       1.00      1.00      1.00      6131
           4       1.00      1.00      1.00      5842
           5       1.00      1.00      1.00      5421
           6       1.00      1.00      1.00      5918
           7       1.00      1.00      1.00      6265
           8       1.00      1.00      1.00      5851
           9       1.00      1.00      1.00      5949

    accuracy                           1.00     60000
   macro avg       1.00      1.00      1.00     60000
weighted avg       1.00      1.00      1.00     60000



In [13]:
predictions = model.predict(X_test)
print(classification_report(y_test.argmax(axis=1), predictions.argmax(axis=1), target_names=[str(x) for x in lb.classes_]))

313/313 [==============================] - 4s 12ms/step
              precision    recall  f1-score   support

           0       1.00      0.99      0.99       980
           1       1.00      1.00      1.00      1135
           2       0.99      1.00      0.99      1032
           3       1.00      1.00      1.00      1010
           4       1.00      0.99      1.00       982
           5       0.99      0.99      0.99       892
           6       1.00      0.99      0.99       958
           7       1.00      0.99      0.99      1028
           8       0.99      1.00      1.00       974
           9       0.99      0.99      0.99      1009

    accuracy                           0.99     10000
   macro avg       0.99      0.99      0.99     10000
weighted avg       0.99      0.99      0.99     10000

